**Khaled Atef<br>
khaledatef7070@gmail.com<br>
https://www.linkedin.com/in/khaledatef1/**

In [1]:
from bs4 import BeautifulSoup
import requests
import unicodedata
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Trying to Scrap One Product only

In [2]:
url = 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/EDEKA-Bio-Fettarme-H-Milch-1-5-1L.html'
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options = options)
driver.get(url)

In [3]:
p = '//*[@id="wrapper"]/div[5]/div/div[2]/div[1]/div[2]/h1'
p_name = driver.find_element(By.XPATH, p).text
p_name

'EDEKA Bio Fettarme H-Milch 1,5% 1L'

In [4]:
p_brand = p_name.split(' ')[0]
p_brand

'EDEKA'

In [5]:
p= '//*[@id="description"]'
p_desc = driver.find_element(By.XPATH ,p).text.strip().replace('\n', '')

p_desc

'Bio Fettarme H-Milch, 1,5% Fett, ultrahocherhitzt, homogenisiertultrahocherhitzthomogenisiert1,5% Fettaus kontrolliert ökologischer LandwirtschaftHinweis zur Aufbewahrung: Nach dem Öffnen kühl lagern und alsbaldig aufbrauchen.Verantwortliches Lebensmittelunternehmen:EDEKA ZENTRALE Stiftung & Co. KG, D-22291 HamburgÖko-Kontrollstellencode: AT-BIO-902'

In [6]:
p='//*[@id="wrapper"]/div[5]/div/div[2]/div[1]/div[1]/div/img'
p_img =  driver.find_element(By.XPATH ,p).get_attribute('src')
p_img


'https://www.edeka24.de/out/pictures/generated/product/1/540_540_90/bio_edeka_hmilch_fettarm_15_1l.jpg'

In [7]:
p = '//*[@id="wrapper"]/div[4]/div/ul/li[3]/a'
p_categeory = driver.find_element(By.XPATH ,p).text
p_categeory

'Kaffee & Tee'

In [8]:
p = '//*[@id="jq_widgetContainer_articleDetailsPrice"]/div[1]/div'
p_price = driver.find_element(By.XPATH ,p).text
p_price

'1,29 €'

In [9]:
options = Options()
options.add_argument("--headless=new")

def get_rate_reviews(url):


    driver = webdriver.Chrome(options = options)
    driver.get(url)
    wait = WebDriverWait(driver, 5)

    p ='//*[@id="tsItemRatings"]/div/div[2]'
    element = wait.until(EC.presence_of_element_located((By.XPATH, p)))
    dynamic_content = driver.find_element(By.XPATH, p)
    prod_rate = dynamic_content.text.split('/')[0]
    
    wait = WebDriverWait(driver, 5)
    p ='//*[@id="trustedshops-productreviews-sticker-wrapper"]/div[1]/div[2]/ul'
    ul = wait.until(EC.presence_of_element_located((By.XPATH, p)))
    inner_html = ul.get_attribute('innerHTML')
    soup = BeautifulSoup(inner_html, 'html.parser')
    driver.quit()
    
    li = soup.find_all(class_='ts-review-text')
    li = [item.text for item in li]
    li = list(filter(lambda x: x.strip(), li))

    return prod_rate ,li


prod_rating , prod_reviews = get_rate_reviews(url)
print(prod_rating ," - " ,prod_reviews)

5.00  -  []


# Getting All Products Links On Edeka

In [10]:
all_sub_groceries = ['https://www.edeka24.de/Lebensmittel/Kaffee-Tee/' , 'https://www.edeka24.de/Lebensmittel/Getraenke/' ,
                     'https://www.edeka24.de/Lebensmittel/Backen-Desserts/','https://www.edeka24.de/Lebensmittel/Fruehstueck-Snacks/',
                     'https://www.edeka24.de/Lebensmittel/Wuerzmittel-Bruehen/','https://www.edeka24.de/Lebensmittel/Sossen/',
                     'https://www.edeka24.de/Lebensmittel/Beilagen/','https://www.edeka24.de/Lebensmittel/Konserven/',
                     'https://www.edeka24.de/Lebensmittel/Suess-Salzig/','https://www.edeka24.de/Lebensmittel/Fertiggerichte/',
                     'https://www.edeka24.de/Lebensmittel/Internationales/'

                    
                    ]
all_links = []


In [ ]:
for url in all_sub_groceries:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    cnt = int(soup.find(class_='category-amount-articles').span.text)
    cnt = int ( cnt / 30 )
    cnt+=5
    while(cnt>0):
        extra = '?pgNr='+str(cnt)
        temp_url = url + extra
        print(temp_url)
        response = requests.get(temp_url)
        soup = BeautifulSoup(response.content, 'lxml')
        prods = soup.find_all(class_='product-details')
        [all_links.append(item.find('a')['href'].split('?')[0]) for item in prods]
        cnt-=1

In [ ]:
len(all_links)

In [ ]:
#Saving Links 
with open('E:\Omdena\Task1\edeka_scrap\Links.txt', 'w') as f:
    for line in all_links:
        f.write(f"{line}\n")

# Scraping All Grocery Product On Edeka

In [11]:
# Helper Functions

def get_rate_reviews(driver , url):

    wait = WebDriverWait(driver, 5)

    p ='//*[@id="tsItemRatings"]/div/div[2]'
    element = wait.until(EC.presence_of_element_located((By.XPATH, p)))
    dynamic_content = driver.find_element(By.XPATH, p)
    prod_rate = dynamic_content.text.split('/')[0]
    
    wait = WebDriverWait(driver, 5)
    p ='//*[@id="trustedshops-productreviews-sticker-wrapper"]/div[1]/div[2]/ul'
    ul = wait.until(EC.presence_of_element_located((By.XPATH, p)))
    inner_html = ul.get_attribute('innerHTML')
    soup = BeautifulSoup(inner_html, 'html.parser')
    driver.quit()
    
    li = soup.find_all(class_='ts-review-text')
    li = [item.text for item in li]
    li = list(filter(lambda x: x.strip(), li))

    return prod_rate ,li




def scrap_product(url):
    options = Options()
    options.add_argument("--headless=new")
    driver = webdriver.Chrome(options = options)
    driver.get(url)
    p_name = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[5]/div/div[2]/div[1]/div[2]/h1').text
    p_brand = p_name.split(' ')[0]
    p_desc = driver.find_element(By.XPATH ,'//*[@id="description"]').text.strip().replace('\n', '')
    p_img =  driver.find_element(By.XPATH ,'//*[@id="wrapper"]/div[5]/div/div[2]/div[1]/div[1]/div/img').get_attribute('src')
    p_categeory = driver.find_element(By.XPATH ,'//*[@id="wrapper"]/div[4]/div/ul/li[3]/a').text
    p_price = driver.find_element(By.XPATH ,'//*[@id="jq_widgetContainer_articleDetailsPrice"]/div[1]/div').text

    try:
        prod_rate,prod_reviews = get_rate_reviews(driver ,url)
    except:
        prod_rate,prod_reviews = None , None
    driver.quit()
    product={}
    product['PROD_NAME'] = p_name
    product['PROD_BRAND'] = p_brand
    product['PROD_RATINGS'] = prod_rate
    product['PROD_REVIEWS'] = prod_reviews
    product['PROD_LINK'] = url
    product['PROD_IMAGE_URL'] = p_img
    product['PROD_CATEGORY'] = p_categeory
    product['PROD_PRICE'] = p_price
    product['PROD_DESCRIPTION'] = p_desc
    return product



In [12]:
# Test 
scrap_product('https://www.edeka24.de/Lebensmittel/Kaffee-Tee/EDEKA-Bio-Fettarme-H-Milch-1-5-1L.html')

{'PROD_NAME': 'EDEKA Bio Fettarme H-Milch 1,5% 1L',
 'PROD_BRAND': 'EDEKA',
 'PROD_RATINGS': '5.00',
 'PROD_REVIEWS': [],
 'PROD_LINK': 'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/EDEKA-Bio-Fettarme-H-Milch-1-5-1L.html',
 'PROD_IMAGE_URL': 'https://www.edeka24.de/out/pictures/generated/product/1/540_540_90/bio_edeka_hmilch_fettarm_15_1l.jpg',
 'PROD_CATEGORY': 'Kaffee & Tee',
 'PROD_PRICE': '1,29 €',
 'PROD_DESCRIPTION': 'Bio Fettarme H-Milch, 1,5% Fett, ultrahocherhitzt, homogenisiertultrahocherhitzthomogenisiert1,5% Fettaus kontrolliert ökologischer LandwirtschaftHinweis zur Aufbewahrung: Nach dem Öffnen kühl lagern und alsbaldig aufbrauchen.Verantwortliches Lebensmittelunternehmen:EDEKA ZENTRALE Stiftung & Co. KG, D-22291 HamburgÖko-Kontrollstellencode: AT-BIO-902'}

In [32]:
f = open('E:\Omdena\Task1\edeka_scrap\Links.txt', 'r')
content = f.read()
content = content.replace('\n','')
content = content.split('https')
content = content[1:]
content = ['https'+link for link in content]
content[0]

'https://www.edeka24.de/Lebensmittel/Kaffee-Tee/Messmer-Schwarztee-Assam-und-Ceylon-Nr-10-lose-400G.html'

In [20]:
import time
grocery_products_edeka = []
cnt =0 
for url in content:
    try:
        product = scrap_product(url)
        grocery_products_edeka.append(product)
    except:
        print(url)
        
    if(cnt%10 == 0):
        time.sleep(5)
        print(cnt)
    cnt+=1

In [21]:
df = pd.DataFrame(grocery_products_edeka)

In [23]:
df['PROD_RATINGS'].unique()

array([ nan, 5.  , 4.85, 2.  , 4.  , 3.5 , 4.86, 3.75, 4.6 , 4.8 , 3.  ,
       3.67, 4.9 , 1.  , 4.83, 4.95, 4.98, 4.74, 4.64, 4.92, 2.5 , 4.93,
       4.96, 4.5 , 4.67, 4.7 , 4.75, 4.17, 3.33, 4.2 , 4.91, 4.22, 4.89,
       4.42, 1.75, 4.57, 4.77, 4.45, 4.25, 4.88, 4.71, 4.73, 4.44, 4.94,
       4.82, 4.33, 3.2 , 3.4 , 3.6 , 4.78, 4.3 , 4.55, 3.8 , 4.84, 4.4 ,
       3.88, 4.81, 4.07, 3.71, 4.62, 4.14, 3.79, 3.82])

In [24]:
df['PROD_CATEGORY'].unique()

array(['Coffee, tea', 'beverages', 'Baking & Desserts',
       'Breakfast & Snacks', 'sauces', 'Side dishes', 'canned goods',
       'Sweet & Salty', 'ready meals', 'International'], dtype=object)

In [28]:
from googletrans import Translator, constants
from pprint import pprint
def translate(key):
    translator = Translator()
    if(key is None):
        return None
    
    if isinstance(key, list):
        return [translator.translate(i).text for i in key]
    else :
        translation = translator.translate(key)
        return translation.text


In [ ]:
# Translating All German Text Into English
df['PROD_NAME'] = df['PROD_NAME'].apply(lambda x: translate(x))
df['PROD_REVIEWS'] = df['PROD_REVIEWS'].apply(lambda x: translate(x))
df['PROD_CATEGORY'] = df['PROD_CATEGORY'].apply(lambda x: translate(x))
df['PROD_DESCRIPTION'] = df['PROD_DESCRIPTION'].apply(lambda x: translate(x))

df['STORE_NAME'] = 'EDEKA'
df['STORE_ID'] = None
df['PROD_POPULARITY'] = None
df['TEAM_MEMBER'] = 'Khaled Atef'

In [ ]:
df.to_csv("edeka_groceries_data_final.csv")